In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00


In [ ]:
import math
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import ResGatedGraphConv
from torch.nn import Linear, Sequential, ReLU

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
PARAMS = {
    'hidden_dim': 16,
    'batch_size': 256,
    'epochs': 200,
    'lr': 0.01,
    'weight_decay': 5e-4,
    'seed': 48  # 0, 13, 16, 42, 48
}

class Params:
    def __init__(self, obj):
        for k, v in obj.items():
            setattr(self, k, v)

params = Params(PARAMS)

In [ ]:
torch.manual_seed(params.seed)

class ResGraphNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ResGraphNet, self).__init__()
        self.conv1 = ResGatedGraphConv(input_dim, hidden_dim)
        self.conv2 = ResGatedGraphConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = ResGraphNet(dataset.num_features, params.hidden_dim, dataset.num_classes)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=params.lr, weight_decay=params.weight_decay)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits, accs = model(data.x, data.edge_index, data.edge_attr), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

best_val_acc = 0
best_test_acc = 0
for epoch in range(1, params.epochs):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      best_test_acc = test_acc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
print("Best test acc: ", best_test_acc)

Epoch: 001, Loss: 2.0073, Train Acc: 0.1429, Val Acc: 0.1620, Test Acc: 0.1520
Epoch: 002, Loss: 1.9878, Train Acc: 0.2000, Val Acc: 0.2100, Test Acc: 0.2040
Epoch: 003, Loss: 1.8814, Train Acc: 0.3286, Val Acc: 0.2920, Test Acc: 0.2700
Epoch: 004, Loss: 1.8607, Train Acc: 0.3571, Val Acc: 0.2780, Test Acc: 0.2760
Epoch: 005, Loss: 1.7753, Train Acc: 0.3857, Val Acc: 0.2980, Test Acc: 0.3100
Epoch: 006, Loss: 1.7397, Train Acc: 0.4571, Val Acc: 0.3580, Test Acc: 0.4140
Epoch: 007, Loss: 1.7547, Train Acc: 0.4786, Val Acc: 0.3900, Test Acc: 0.4230
Epoch: 008, Loss: 1.7530, Train Acc: 0.4929, Val Acc: 0.4000, Test Acc: 0.4350
Epoch: 009, Loss: 1.7782, Train Acc: 0.4786, Val Acc: 0.4300, Test Acc: 0.4470
Epoch: 010, Loss: 1.7415, Train Acc: 0.5143, Val Acc: 0.4560, Test Acc: 0.4690
Epoch: 011, Loss: 1.6274, Train Acc: 0.5357, Val Acc: 0.4640, Test Acc: 0.4960
Epoch: 012, Loss: 1.6483, Train Acc: 0.5500, Val Acc: 0.5020, Test Acc: 0.5220
Epoch: 013, Loss: 1.6370, Train Acc: 0.5857, Val Acc